In [1]:
import pymc3 as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from theano import tensor as tt
import scipy.stats as st
import seaborn as sns
import scipy
import import_ipynb
import GP_util #load module first
import importlib
importlib.reload(GP_util) # reload full module
from GP_util import unique, select, distancematrix,linear_pixel
#import methods from reloaded module

/home/brian/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


importing Jupyter notebook from GP_util.ipynb
importing Jupyter notebook from GP_util.ipynb


In [2]:
datapath = '/home/brian/WesternWheat/Data/Processed/maxdf_24.csv'
df = pd.read_csv(datapath)

df = select(df, n=200) # selects subset of df for time/memory reductions
f, df_unique, zero_farm_idx, Num_farms = unique(df)

there are 194 unique farms in loaded df


In [4]:
def fastdistancematrix(df, distance_calc=True, sparse=False, dlim=100):
    """
    inputs: df

    returns:
    1.) distance between all farms in miles
    2.) distance^2

    """


    from scipy.spatial import distance_matrix
    from geopy.distance import geodesic
    unique_farms = pd.unique(df.pixel)
    distance = np.zeros((unique_farms.size,unique_farms.size))
    df_unique = df.set_index('pixel')
    df_unique = df_unique[~df_unique.index.duplicated(keep='first')] # only keep unique index values

    for i in range(unique_farms.size):
    lat_lon_i = df_unique.Latitude.iloc[i],df_unique.Longitude.iloc[i]
    for j in range(i):
        lat_lon_j = df_unique.Latitude.iloc[j],df_unique.Longitude.iloc[j]
        if distance_calc == True:
            distance[i,j] = geodesic(lat_lon_i, lat_lon_j).miles
            distance[j,i] = distance[i,j] # make use of symmetry
    return distance, np.power(distance, 2)

distance, Dmatsq = fastdistancematrix(df)

In [5]:
d,dmatsq = distancematrix(df)

In [23]:
# fast version does indeed work
np.where(distance == d)[1].size, Num_farms**2

(37636, 37636)